# Strokes Gained Model

In [19]:
import requests
from bs4 import BeautifulSoup
url_stub = "http://www.pgatour.com/stats/stat.%s.%s.html" #stat id, year
category_url_stub = 'http://www.pgatour.com/stats/categories.%s.html'
category_labels = ['RPTS_INQ', 'ROTT_INQ', 'RAPP_INQ', 'RARG_INQ', 'RPUT_INQ', 'RSCR_INQ', 'RSTR_INQ', 'RMNY_INQ']
pga_tour_base_url = "http://www.pgatour.com"
def gather_pages(url, filename):
    print(filename)
    urllib.urlretrieve(url, filename)

def gather_html():
    stat_ids = []
    for category in category_labels:
        category_url = category_url_stub % (category)
        page = requests.get(category_url)
        html = BeautifulSoup(page.text.replace('\n',''), 'html.parser')
    for table in html.find_all("div", class_="table-content"):
        for link in table.find_all("a"):
             stat_ids.append(link['href'].split('.')[1])

    starting_year = 2015 #page in order to see which years we have info for
    for stat_id in stat_ids:
        url = url_stub % (stat_id, starting_year)
        page = requests.get(url)
        html = BeautifulSoup(page.text.replace('\n',''), 'html.parser')
        print("HTML: ",html)
        stat = html.find("div", class_="parsys mainParsys").find('h3').text
        #print(stat)
        directory = "stats_html/%s" % stat.replace('/', ' ') #need to replace to avoid
        if not os.path.exists(directory):
            os.makedirs(directory)
        years = []
        for option in html.find("select", class_="statistics-details-select").find_all("option"):
            year = option['value']
            if year not in years:
                years.append(year)
        url_filenames = []
        for year in years:
            url = url_stub % (stat_id, year)
            filename = "%s/%s.html" % (directory, year)
            if not os.path.isfile(filename): #this check saves time if you've already downloaded the page
                url_filenames.append((url, filename))
                jobs = [gevent.spawn(gather_pages, pair[0], pair[1]) for pair in url_filenames]
                gevent.joinall(jobs)

In [20]:
gather_html()

HTML:  <!DOCTYPE HTML>
</script><script>        window.__gigyaConf = {            cid: window.location.hostname.replace('www.', '').toUpperCase()        }    </script> <script src="https://cdns.gigya.com/JS/socialize.js?apikey=3_IscKmAoYcuwP8zpTnatC3hXBUm8rPuI-Hg_cZJ-jL-M7LgqCkxmwe-ps1Qy7PoWd" type="text/javascript"></script> <script type="text/javascript">        pgatour.GigyaSocialize.init({            getIpUrl: 'https://prod.livefyre.ixsebu.com/ip',            kruxBaseUrl: '//beacon.krxd.net/usermatch.gif?',            kruxPartnerId: 'gigya',            playersFeed: 'https://statdata.pgatour.com/players/player.json'        });    </script><script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"errorBeacon":"bam-cell.nr-data.net","licenseKey":"0ee18552e2","agent":"","beacon":"bam-cell.nr-data.net","applicationTime":1285,"applicationID":"686519500","transactionName":"ZVRRbUBVWhUHBUVfDVwecExBQFsLSQdBRhEdQVRYRltBFEsVWEIHHUVWVEJYVRIDFR5FFlNFd1xGVV0KFTZQUQc=","queueTime":0}</

AttributeError: 'NoneType' object has no attribute 'find'